# Setup 

Download the Task data and evaluation scripts

In [1]:
!git clone https://github.com/H-TayyarMadabushi/SemEval_2022_Task2-idiomaticity.git

Cloning into 'SemEval_2022_Task2-idiomaticity'...
remote: Enumerating objects: 92, done.
remote: Counting objects: 100% (92/92), done.
remote: Compressing objects: 100% (75/75), done.
remote: Total 92 (delta 31), reused 67 (delta 15), pack-reused 0
Unpacking objects: 100% (92/92), done.


Download the “AStitchInLanguageModels” code which we make use of. 

In [2]:
!git clone https://github.com/H-TayyarMadabushi/AStitchInLanguageModels.git

Cloning into 'AStitchInLanguageModels'...
remote: Enumerating objects: 1030, done.
remote: Counting objects: 100% (1030/1030), done.
remote: Compressing objects: 100% (772/772), done.
remote: Total 1030 (delta 382), reused 803 (delta 202), pack-reused 0
Receiving objects: 100% (1030/1030), 79.86 MiB | 13.34 MiB/s, done.
Resolving deltas: 100% (382/382), done.


Download and install an editable version of huggingfaces transformers. 

In [3]:
!git clone https://github.com/huggingface/transformers.git
%cd transformers/
!pip install --editable .
%cd /content/ 

Cloning into 'transformers'...
remote: Enumerating objects: 91488, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 91488 (delta 0), reused 1 (delta 0), pack-reused 91483
Receiving objects: 100% (91488/91488), 75.67 MiB | 14.39 MiB/s, done.
Resolving deltas: 100% (66031/66031), done.
/content/transformers
Obtaining file:///content/transformers
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 61 kB 501 kB/s 
     |████████████████████████████████| 596 kB 8.5 MB/s 
     |████████████████████████████████| 895 kB 48.5 MB/s 
     |████████████████████████████████| 3.3 MB 26.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Running setup.py develop for transformers
/content


Required for run_glue ... 

In [4]:
## run_glue needs this. 
!pip install datasets

     |████████████████████████████████| 298 kB 4.0 MB/s 
     |████████████████████████████████| 132 kB 68.4 MB/s 
     |████████████████████████████████| 243 kB 53.3 MB/s 
     |████████████████████████████████| 1.1 MB 56.3 MB/s 
     |████████████████████████████████| 271 kB 67.3 MB/s 
     |████████████████████████████████| 160 kB 70.6 MB/s 
     |████████████████████████████████| 192 kB 53.5 MB/s 


Editable install requires runtime restart unless we do this. 

In [5]:
import site
site.main()


# Imports and Helper functions

In [6]:
import os
import csv

from pathlib import Path

In [7]:
def load_csv( path, delimiter=',' ) : 
  header = None
  data   = list()
  with open( path, encoding='utf-8') as csvfile:
    reader = csv.reader( csvfile, delimiter=delimiter ) 
    for row in reader : 
      if header is None : 
        header = row
        continue
      data.append( row ) 
  return header, data


In [8]:
def write_csv( data, location ) : 
  with open( location, 'w', encoding='utf-8') as csvfile:
    writer = csv.writer( csvfile ) 
    writer.writerows( data ) 
  print( "Wrote {}".format( location ) ) 
  return


The following function creates a submission file from the predictions output by run_glue (the text classification script from huggingface transformers - see below). 

Note that we set it up so we can load up results for only one setting. 

It requires as input the submission format file, which is available with the data. You can call this after completing each setting to load up results for both settings (see below).


In [9]:
def insert_to_submission_file( submission_format_file, input_file, prediction_format_file, setting ) :
    submission_header, submission_content = load_csv( submission_format_file )
    input_header     , input_data         = load_csv( input_file             )
    prediction_header, prediction_data    = load_csv( prediction_format_file, '\t' )

    assert len( input_data ) == len( prediction_data )

    ## submission_header ['ID', 'Language', 'Setting', 'Label']
    ## input_header      ['label', 'sentence1' ]
    ## prediction_header ['index', 'prediction']

    prediction_data = list( reversed( prediction_data ) )

    started_insert  = False
    for elem in submission_content : 
        if elem[ submission_header.index( 'Setting' ) ] != setting :
            if started_insert :
                if len( prediction_data ) == 0 :
                    break
                else : 
                    raise Exception( "Update should to contiguous ... something wrong." ) 
            continue
        started_insert = True
        elem[ submission_header.index( 'Label' ) ] = prediction_data.pop()[ prediction_header.index( 'prediction' ) ]

    return [ submission_header ] + submission_content

In [10]:
def _get_train_data( data_location, file_name, include_context, include_idiom ) :
    
    file_name = os.path.join( data_location, file_name ) 

    header, data = load_csv( file_name )

    out_header = [ 'label', 'sentence1' ]
    if include_idiom :
        out_header = [ 'label', 'sentence1', 'sentence2' ]
        
    # ['DataID', 'Language', 'MWE', 'Setting', 'Previous', 'Target', 'Next', 'Label']
    out_data = list()
    for elem in data :
        label     = elem[ header.index( 'Label'  ) ]
        sentence1 = elem[ header.index( 'Target' ) ]
        if include_context :
            sentence1 = ' '.join( [ elem[ header.index( 'Previous' ) ], elem[ header.index( 'Target' ) ], elem[ header.index( 'Next' ) ] ] )
        this_row = None
        if not include_idiom :
            this_row = [ label, sentence1 ] 
        else :
            sentence2 = elem[ header.index( 'MWE' ) ]
            this_row = [ label, sentence1, sentence2 ]
        out_data.append( this_row )
        assert len( out_header ) == len( this_row )
    return [ out_header ] + out_data

In [11]:
def _get_dev_eval_data( data_location, input_file_name, gold_file_name, include_context, include_idiom ) :

    input_headers, input_data = load_csv( os.path.join( data_location, input_file_name ) )
    gold_header  = gold_data = None
    if not gold_file_name is None : 
        gold_header  , gold_data  = load_csv( os.path.join( data_location, gold_file_name  ) )
        assert len( input_data ) == len( gold_data )

    # ['ID', 'Language', 'MWE', 'Previous', 'Target', 'Next']
    # ['ID', 'DataID', 'Language', 'Label']
    
    out_header = [ 'label', 'sentence1' ]
    if include_idiom :
        out_header = [ 'label', 'sentence1', 'sentence2' ]

    out_data = list()
    for index in range( len( input_data ) ) :
        label = 1
        if not gold_file_name is None : 
            this_input_id = input_data[ index ][ input_headers.index( 'ID' ) ]
            this_gold_id  = gold_data [ index ][ gold_header  .index( 'ID' ) ]
            assert this_input_id == this_gold_id
            
            label     = gold_data[ index ][ gold_header.index( 'Label'  ) ]
            
        elem      = input_data[ index ]
        sentence1 = elem[ input_headers.index( 'Target' ) ]
        if include_context :
            sentence1 = ' '.join( [ elem[ input_headers.index( 'Previous' ) ], elem[ input_headers.index( 'Target' ) ], elem[ input_headers.index( 'Next' ) ] ] )
        this_row = None
        if not include_idiom :
            this_row = [ label, sentence1 ] 
        else :
            sentence2 = elem[ input_headers.index( 'MWE' ) ]
            this_row = [ label, sentence1, sentence2 ]
        assert len( out_header ) == len( this_row ) 
        out_data.append( this_row )
        

    return [ out_header ] + out_data


In [12]:
"""
Based on the results presented in `AStitchInLanguageModels' we work with not including the idiom for the zero shot setting and including it in the one shot setting.
"""
def create_data( input_location, output_location ) :

    
    ## Zero shot data
    train_data = _get_train_data(
        data_location   = input_location,
        file_name       = 'train_zero_shot.csv',
        include_context = True,
        include_idiom   = False
    )
    write_csv( train_data, os.path.join( output_location, 'ZeroShot', 'train.csv' ) )
    
    dev_data = _get_dev_eval_data(
        data_location    = input_location,
        input_file_name  = 'dev.csv',
        gold_file_name   = 'dev_gold.csv', 
        include_context  = True,
        include_idiom    = False
    )        
    write_csv( dev_data, os.path.join( output_location, 'ZeroShot', 'dev.csv' ) )
    
    eval_data = _get_dev_eval_data(
        data_location    = input_location,
        input_file_name  = 'eval.csv',
        gold_file_name   = None , ## Don't have gold evaluation file -- submit to CodaLab
        include_context  = True,
        include_idiom    = False
    )
    write_csv( eval_data, os.path.join( output_location, 'ZeroShot', 'eval.csv' ) )


    ## OneShot Data (combine both for training)
    train_zero_data = _get_train_data(
        data_location   = input_location,
        file_name       = 'train_zero_shot.csv',
        include_context = False,
        include_idiom   = True
    )
    train_one_data = _get_train_data(
        data_location   = input_location,
        file_name       = 'train_one_shot.csv',
        include_context = False,
        include_idiom   = True
    )

    assert train_zero_data[0] == train_one_data[0] ## Headers
    train_data = train_one_data + train_zero_data[1:]
    write_csv( train_data, os.path.join( output_location, 'OneShot', 'train.csv' ) )
    
    dev_data = _get_dev_eval_data(
        data_location    = input_location,
        input_file_name  = 'dev.csv',
        gold_file_name   = 'dev_gold.csv', 
        include_context  = False,
        include_idiom    = True
    )        
    write_csv( dev_data, os.path.join( output_location, 'OneShot', 'dev.csv' ) )
    
    eval_data = _get_dev_eval_data(
        data_location    = input_location,
        input_file_name  = 'eval.csv',
        gold_file_name   = None,
        include_context  = False,
        include_idiom    = True
    )
    write_csv( eval_data, os.path.join( output_location, 'OneShot', 'eval.csv' ) )

    return

In [13]:
import pandas as pd
df = pd.read_csv("/content/SemEval_2022_Task2-idiomaticity/SubTaskA/Data/train_zero_shot.csv")
df

,DataID,Language,MWE,Setting,Previous,Target,Next,Label
0,train_zero_shot.EN.168.1,EN,double dutch,zero_shot,This inspired others to jump ropes as a leisur...,There are several theories behind the origin o...,The most popular theory states that “Double Du...,0
1,train_zero_shot.EN.168.2,EN,double dutch,zero_shot,In the age of chivalry a man paid for the woma...,"Double Dutch also derives from the same era, D...",There are many phrases that include the word: ...,0
2,train_zero_shot.EN.168.3,EN,double dutch,zero_shot,"To her eternal credit, she kept both India and...",Since 1977 we have had a plethora of Foreign M...,We need to exclude from that list the late Mr ...,0
3,train_zero_shot.EN.168.4,EN,double dutch,zero_shot,While pharmaceutical companies were researchin...,Turns out that these people were speaking doub...,So why aren’t Big Macs sold all over the world...,0
4,train_zero_shot.EN.168.5,EN,double dutch,zero_shot,Coronavirus in Europe * Brexit * Brussels ...,Is Flemish premier talking double Dutch?,Three months before the Belgians take over the...,0
...,...,...,...,...,...,...,...,...
4486,train_zero_shot.PT.351.8,PT,gato-pingado,zero_shot,Era o ano de 1968.,"A estação de passageiros tranquila, com um ou...","Com os primeiros raios solares, o potente DC 8...",0
4487,train_zero_shot.PT.351.9,PT,gato-pingado,zero_shot,De acordo com informações vazadas de dentro da...,"Segundo a fonte, o programa vai seguir um form...","Leonardo fica inchado, perde a voz e vício mor...",0
4488,train_zero_shot.PT.351.10,PT,gato-pingado,zero_shot,O radialista Roberto Toledo apresentou o show ...,"""Eu fiquei sentado no chão e realmente só tinh...",Entre os poucos espectadores estava o advogado...,0
4489,train_zero_shot.PT.351.11,PT,gato-pingado,zero_shot,O carnaval em Goiânia também tem espaço para o...,"É o caso do bloco “Gato Pingado”, que reuniu c...","Segundo os organizadores, quando criaram o blo...",1


In [14]:
outpath = 'Data'
    
Path( os.path.join( outpath, 'ZeroShot' ) ).mkdir(parents=True, exist_ok=True)
Path( os.path.join( outpath, 'OneShot' ) ).mkdir(parents=True, exist_ok=True)

create_data( 'SemEval_2022_Task2-idiomaticity/SubTaskA/Data/', outpath )

Wrote Data/ZeroShot/train.csv
Wrote Data/ZeroShot/dev.csv
Wrote Data/ZeroShot/eval.csv
Wrote Data/OneShot/train.csv
Wrote Data/OneShot/dev.csv
Wrote Data/OneShot/eval.csv


Load Model Here

In [ ]:
from transformers import BartForSequenceClassification, BartTokenizer, BartConfig
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-mnli')
model = BartForSequenceClassification.from_pretrained('facebook/bart-large-mnli')

In [16]:

df_eval= pd.read_csv("Data/ZeroShot/dev.csv")

In [40]:
df_eval.sentence1[0]

'Does the plumbing predictably rebel, creating a 1,000-foot cascade inside the central utility shaft? Are these interruptions of the good life a necessary condition of the high life? Not at all , says James von Klemperer, president of the architecture firm Kohn Pedersen Fox, which plants skyscrapers all over the world: “In a building that thin, this kind of thing can happen, but it shouldn’t.'

In [20]:
model.to(torch.device('cuda:0'))

BartForSequenceClassification(
  (model): BartModel(
    (shared): Embedding(50265, 1024, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(50265, 1024, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0): BartEncoderLayer(
          (self_attn): BartAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (final_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
  

Hypothesis Based Approach

In [35]:
from sklearn.metrics import classification_report
import numpy as np
import torch
def find_class_idiom(mnli_model, mnli_tokenizer, labels, list_of_text):
  hypothesis = 'idiomatic'
  preds = []

  for premise in list_of_text:
    input_ids = mnli_tokenizer.encode(premise, hypothesis, return_tensors='pt').to(torch.device('cuda:0'))
    logits = mnli_model(input_ids)[0]
    entail_contradiction_logits = logits[:,[0, 2]]
    probs = entail_contradiction_logits.softmax(dim=1)
    
    true_prob = probs[:,1].item() * 100
    if (true_prob >= 50):
      preds.append(0)
    else:
      preds.append(1)
    print(f'Probability that the label is true: {true_prob:0.2f}%')
  target_names = ['Idiomatic', 'Non-Idiomatic']
  print(classification_report(labels, preds, target_names=target_names))

In [36]:
find_class_idiom(model, tokenizer, df_eval.label, df_eval.sentence1)

Probability that the label is true: 4.04%
Probability that the label is true: 43.44%
Probability that the label is true: 18.06%
Probability that the label is true: 27.10%
Probability that the label is true: 31.84%
Probability that the label is true: 81.39%
Probability that the label is true: 58.86%
Probability that the label is true: 60.69%
Probability that the label is true: 74.56%
Probability that the label is true: 58.24%
Probability that the label is true: 34.85%
Probability that the label is true: 3.87%
Probability that the label is true: 5.91%
Probability that the label is true: 35.79%
Probability that the label is true: 34.03%
Probability that the label is true: 24.10%
Probability that the label is true: 46.77%
Probability that the label is true: 48.66%
Probability that the label is true: 39.76%
Probability that the label is true: 24.58%
Probability that the label is true: 26.59%
Probability that the label is true: 34.18%
Probability that the label is true: 35.32%
Probability th